In [52]:
from dotenv import load_dotenv
import os

load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [53]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [54]:
import ast

def extract_definitions(code):
    tree = ast.parse(code)
    functions = []
    classes = []

    # preOrder traversal of the tree
    for node in tree.body:
        if isinstance(node, ast.FunctionDef):
            functions.append((node.name, ast.get_source_segment(code, node)))
        elif isinstance(node, ast.ClassDef):
            classes.append((node.name, ast.get_source_segment(code, node)))
    return functions, classes


In [55]:
def summarize_function(function_name, function_code):
    prompt = f"""You are a Python code documentation assistant.

Please explain the following function in **Markdown format**, following the structure below:

### Function: `{function_name}`

```python
{function_code}
```

### Summary

[Your detailed explanation here]

### Key Points
- [Point 1]

"""

    response = client.models.generate_content_stream(
        model="gemini-2.5-flash",
        contents=prompt
    )

    text = ""

    for stream in response:
        if stream.text is not None:
            text += stream.text
            print(stream.text, end="", flush=True)
        else:
            print("No text in stream", flush=True)
    return text

In [56]:
def summarize_class(class_name, class_code):
    prompt = f"""You are a code summarizer. Please explain the following class in detail using Markdown format.

### Class: `{class_name}`

```python
{class_code}
```
### Summary

[Your detailed explanation here]

### Key Points
- [Point 1]

"""

    response = client.models.generate_content_stream(
        model="gemini-2.5-flash",
        contents=prompt
    )

    text = ""
    
    for stream in response:
        if stream.text is not None:
            text += stream.text
            print(stream.text, end="", flush=True)
        else:
            print("No text in stream", flush=True)
    return text

In [57]:
def summarize_file(filepath):
    with open(filepath, "r") as file:
        code = file.read()

    functions, classes = extract_definitions(code)
    summary = []

    for name, func_code in functions:
        func_summary = summarize_function(name, func_code)
        summary.append(f"Function `{name}`:\n{func_summary}\n")

    for name, class_code in classes:
        class_summary = summarize_class(name, class_code)
        summary.append(f"Class `{name}`:\n{class_summary}\n")

    return "\n\n---\n\n".join(summary)


In [58]:
def save_summary(summary_text, output_path):
    with open(output_path, "w") as f:
        f.write(summary_text)

In [59]:
if __name__ == "__main__":
    input_path = "code.py"
    output_path = "summary.md"
    
    summary = summarize_file(input_path)
    save_summary(summary, output_path)
    print(f"Summary saved to {output_path}")

### Function: `greet`

```python
def greet(name):
    print(f"Hello, {name}!")

    def temp():
        print("temp")
```

### Summary

The `greet` function is designed to print a personalized greeting message to the console. It takes a single argument, `name`, and embeds it into a "Hello, {name}!" string using an f-string before printing.

An interesting aspect of this function is the definition of a nested function named `temp` within its scope. While `temp` is defined to print the string "temp", it is **never called** from within the `greet` function. This means `temp` exists as a callable object within `greet`'s execution context, but its code will not run when `greet` is invoked unless explicitly called by `greet` or returned and called externally.

### Key Points
- **Purpose:** Prints a "Hello, [name]!" message to standard output.
- **Parameters:** Accepts one argument, `name`, which is expected to be a string.
- **Side Effects:** Directly prints output to the console.
- **Nested

In [60]:
with open("code.py", "r") as file:
    code = file.read()
    tree = ast.parse(code)

    for node in tree.body:
        print(node)
        print(node.__dict__)
        print([child for child in ast.iter_child_nodes(node)])
        print("")


{'name': 'Calculator', 'bases': [], 'keywords': [], 'body': [<ast.FunctionDef object at 0x0000021A0A65E550>, <ast.FunctionDef object at 0x0000021A0A65FD10>, <ast.FunctionDef object at 0x0000021A0A65F390>, <ast.FunctionDef object at 0x0000021A0B26CC90>, <ast.FunctionDef object at 0x0000021A0BE8E450>, <ast.FunctionDef object at 0x0000021A0B1F7990>], 'decorator_list': [], 'type_params': [], 'lineno': 1, 'col_offset': 0, 'end_lineno': 18, 'end_col_offset': 26}
[<ast.FunctionDef object at 0x0000021A0A65E550>, <ast.FunctionDef object at 0x0000021A0A65FD10>, <ast.FunctionDef object at 0x0000021A0A65F390>, <ast.FunctionDef object at 0x0000021A0B26CC90>, <ast.FunctionDef object at 0x0000021A0BE8E450>, <ast.FunctionDef object at 0x0000021A0B1F7990>]

{'name': 'greet', 'args': <ast.arguments object at 0x0000021A0B1F7410>, 'body': [<ast.Expr object at 0x0000021A0B1F7190>, <ast.FunctionDef object at 0x0000021A0B1F6CD0>], 'decorator_list': [], 'returns': None, 'type_comment': None, 'type_params': []

In [61]:
import ast

with open("code.py", "r") as file:
    code = file.read()

    functions, classes = extract_definitions(code)

    for name, func_code in functions:
        print(func_code)

    for name, class_code in classes:
        print(class_code)



def greet(name):
    print(f"Hello, {name}!")

    def temp():
        print("temp")
class Calculator:
    def __init__(self):
        self.result = 0

    def add(self, a, b):
        self.result = a + b

    def subtract(self, a, b):
        self.result = a - b

    def multiply(self, a, b):
        self.result = a * b

    def divide(self, a, b):
        self.result = a / b

    def get_result(self):
        return self.result
class Animal:
    def __init__(self, name):
        self.name = name
